In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/business-json/Task_3_ner_results'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/business-json/Task_3_ner_results/Task_3_ner_results/Retrieved_sentences/Task3_CLIP_label_retrieved_sentences_NER_results.csv
/kaggle/input/business-json/Task_3_ner_results/Task_3_ner_results/Retrieved_sentences/Task3_CLIP_full_Knowledge_retrieved_sentences_NER_results.csv
/kaggle/input/business-json/Task_3_ner_results/Task_3_ner_results/Retrieved_sentences/Task3_CLIP_partial_Knowledge_retrieved_sentences_NER_results.csv
/kaggle/input/business-json/Task_3_ner_results/Task_3_ner_results/Positive_sentences/Task3_CLIP_label_positive_sentences_NER_results.csv
/kaggle/input/business-json/Task_3_ner_results/Task_3_ner_results/Positive_sentences/Task3_CLIP_partial_knowledge_positive_sentences_NER_results.csv
/kaggle/input/business-json/Task_3_ner_results/Task_3_ner_results/Positive_sentences/Task3_CLIP_full_knowledge_positive_sentences_NER_results (1).csv


In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_ner_performance_samplewise_positive(data_df_path, ner_df_path, title):
    # Read the CSV files
    data_df = pd.read_csv(data_df_path)
    ner_df = pd.read_csv(ner_df_path)
    
    # Dictionary to store metrics for each QID
    metrics_per_qid = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}
    
    # Iterate over each QID in data_df
    for idx, row in data_df.iterrows():
        qid = row['QID']
        positive_sentence_ids = eval(row['positive_sentence_ids'])  # Convert stringified list to list
        entities = eval(row['entities'])  # Convert stringified list to list
        
        # Filter ner_df for the same q_id
        ner_subset = ner_df[ner_df['q_id'] == qid]
        
        # Initialize lists to keep track of true labels and predictions for this QID
        true_labels = []
        predictions = []
        
        # Check if any positive_sentence_ids are in retrieved_sentence_ids
        for i, pos_sent_id in enumerate(positive_sentence_ids):
            # Check if pos_sent_id exists in retrieved_sentence_ids of ner_df
            match = ner_subset[ner_subset['positive_sentence_ids'].apply(lambda x: pos_sent_id in eval(x))]
            
            if not match.empty:
                retrieved_index = match.index[0]  # Get the first matching row
                top_entities = eval(match.loc[retrieved_index, 'top_entities'])
                
                # Compare the entity from data_df with the top entities from ner_df
                if entities[i] in top_entities:
                    true_labels.append(1)
                    predictions.append(1)
                else:
                    true_labels.append(1)
                    predictions.append(0)
            else:
                # If no match found, it means the entity is not retrieved correctly
                true_labels.append(1)
                predictions.append(0)
        
        # Calculate metrics for this QID
        accuracy = accuracy_score(true_labels, predictions)
        precision = precision_score(true_labels, predictions, zero_division=0)
        recall = recall_score(true_labels, predictions, zero_division=0)
        f1 = f1_score(true_labels, predictions, zero_division=0)
        
        # Store the results in the metrics dictionary
        metrics_per_qid['accuracy'].append(accuracy)
        metrics_per_qid['precision'].append(precision)
        metrics_per_qid['recall'].append(recall)
        metrics_per_qid['f1'].append(f1)
        
#         # Print the metrics for this specific QID
#         print(f"Metrics for QID {qid}:")
#         print(f"  Accuracy: {accuracy:.4f}")
#         print(f"  Precision: {precision:.4f}")
#         print(f"  Recall: {recall:.4f}")
#         print(f"  F1 Score: {f1:.4f}\n")
    
    # Compute average of each metric
    avg_accuracy = sum(metrics_per_qid['accuracy']) / len(metrics_per_qid['accuracy'])
    avg_precision = sum(metrics_per_qid['precision']) / len(metrics_per_qid['precision'])
    avg_recall = sum(metrics_per_qid['recall']) / len(metrics_per_qid['recall'])
    avg_f1 = sum(metrics_per_qid['f1']) / len(metrics_per_qid['f1'])
    
    # Output the averaged results with the given title
    print(f"Average Evaluation Results for {title}:")
    print(f"  Average Accuracy: {avg_accuracy:.4f}")
    print(f"  Average Precision: {avg_precision:.4f}")
    print(f"  Average Recall: {avg_recall:.4f}")
    print(f"  Average F1 Score: {avg_f1:.4f}")
    
#     return avg_accuracy, avg_precision, avg_recall, avg_f1


In [3]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_ner_performance_samplewise_retrieved(data_df_path, ner_df_path, title):
    # Read the CSV files
    data_df = pd.read_csv(data_df_path)
    ner_df = pd.read_csv(ner_df_path)
    
    # Dictionary to store metrics for each QID
    metrics_per_qid = {'accuracy': [], 'precision': [], 'recall': [], 'f1': []}
    
    # Iterate over each QID in data_df
    for idx, row in data_df.iterrows():
        qid = row['QID']
        positive_sentence_ids = eval(row['positive_sentence_ids'])  # Convert stringified list to list
        entities = eval(row['entities'])  # Convert stringified list to list
        
        # Filter ner_df for the same q_id
        ner_subset = ner_df[ner_df['q_id'] == qid]
        
        # Initialize lists to keep track of true labels and predictions for this QID
        true_labels = []
        predictions = []
        
        # Check if any positive_sentence_ids are in retrieved_sentence_ids
        for i, pos_sent_id in enumerate(positive_sentence_ids):
            # Check if pos_sent_id exists in retrieved_sentence_ids of ner_df
            match = ner_subset[ner_subset['retrieved_sentence_ids'].apply(lambda x: pos_sent_id in eval(x))]
            
            if not match.empty:
                retrieved_index = match.index[0]  # Get the first matching row
                top_entities = eval(match.loc[retrieved_index, 'top_entities'])
                
                # Compare the entity from data_df with the top entities from ner_df
                if entities[i] in top_entities:
                    true_labels.append(1)
                    predictions.append(1)
                else:
                    true_labels.append(1)
                    predictions.append(0)
            else:
                # If no match found, it means the entity is not retrieved correctly
                true_labels.append(1)
                predictions.append(0)
        
        # Calculate metrics for this QID
        accuracy = accuracy_score(true_labels, predictions)
        precision = precision_score(true_labels, predictions, zero_division=0)
        recall = recall_score(true_labels, predictions, zero_division=0)
        f1 = f1_score(true_labels, predictions, zero_division=0)
        
        # Store the results in the metrics dictionary
        metrics_per_qid['accuracy'].append(accuracy)
        metrics_per_qid['precision'].append(precision)
        metrics_per_qid['recall'].append(recall)
        metrics_per_qid['f1'].append(f1)
        
#         # Print the metrics for this specific QID
#         print(f"Metrics for QID {qid}:")
#         print(f"  Accuracy: {accuracy:.4f}")
#         print(f"  Precision: {precision:.4f}")
#         print(f"  Recall: {recall:.4f}")
#         print(f"  F1 Score: {f1:.4f}\n")
    
    # Compute average of each metric
    avg_accuracy = sum(metrics_per_qid['accuracy']) / len(metrics_per_qid['accuracy'])
    avg_precision = sum(metrics_per_qid['precision']) / len(metrics_per_qid['precision'])
    avg_recall = sum(metrics_per_qid['recall']) / len(metrics_per_qid['recall'])
    avg_f1 = sum(metrics_per_qid['f1']) / len(metrics_per_qid['f1'])
    
    # Output the averaged results with the given title
    print(f"Average Evaluation Results for {title}:")
    print(f"  Average Accuracy: {avg_accuracy:.4f}")
    print(f"  Average Precision: {avg_precision:.4f}")
    print(f"  Average Recall: {avg_recall:.4f}")
    print(f"  Average F1 Score: {avg_f1:.4f}")
    
#     return avg_accuracy, avg_precision, avg_recall, avg_f1


In [4]:
data_df_path = '/kaggle/input/business-json/Task_3_data.csv'
ner_df_path = '/kaggle/input/business-json/Task_3_ner_results/Task_3_ner_results/Positive_sentences/Task3_CLIP_full_knowledge_positive_sentences_NER_results (1).csv'
title = 'NER Evaluation for positive sentences on full_knowledge'

# Call the function and print the results
# accuracy, precision, recall, f1 = evaluate_ner_performance(data_df_path, ner_df_path, title)
evaluate_ner_performance_samplewise_positive(data_df_path, ner_df_path, title)
# # Output the results
# print(f"Accuracy: {accuracy}")
# print(f"Precision: {precision}")
# print(f"Recall: {recall}")
# print(f"F1 Score: {f1}")

Average Evaluation Results for NER Evaluation for positive sentences on full_knowledge:
  Average Accuracy: 0.6121
  Average Precision: 0.8190
  Average Recall: 0.6121
  Average F1 Score: 0.6802


In [5]:
ner_df_path = '/kaggle/input/business-json/Task_3_ner_results/Task_3_ner_results/Positive_sentences/Task3_CLIP_partial_knowledge_positive_sentences_NER_results.csv'
title = 'NER Evaluation for positive sentences on partial_knowledge'

# Call the function and print the results
# accuracy, precision, recall, f1 = evaluate_ner_performance(data_df_path, ner_df_path, title)
evaluate_ner_performance_samplewise_positive(data_df_path, ner_df_path, title)
# # Output the results
# print(f"Accuracy: {accuracy}")
# print(f"Precision: {precision}")
# print(f"Recall: {recall}")
# print(f"F1 Score: {f1}")

Average Evaluation Results for NER Evaluation for positive sentences on partial_knowledge:
  Average Accuracy: 0.6106
  Average Precision: 0.8276
  Average Recall: 0.6106
  Average F1 Score: 0.6825


In [6]:
ner_df_path = '/kaggle/input/business-json/Task_3_ner_results/Task_3_ner_results/Positive_sentences/Task3_CLIP_label_positive_sentences_NER_results.csv'
title = 'NER Evaluation for positive sentences on labels'

# Call the function and print the results
# accuracy, precision, recall, f1 = evaluate_ner_performance(data_df_path, ner_df_path, title)
evaluate_ner_performance_samplewise_positive(data_df_path, ner_df_path, title)
# # Output the results
# print(f"Accuracy: {accuracy}")
# print(f"Precision: {precision}")
# print(f"Recall: {recall}")
# print(f"F1 Score: {f1}")

Average Evaluation Results for NER Evaluation for positive sentences on labels:
  Average Accuracy: 0.5891
  Average Precision: 0.8276
  Average Recall: 0.5891
  Average F1 Score: 0.6672


In [7]:
ner_df_path = '/kaggle/input/business-json/Task_3_ner_results/Task_3_ner_results/Retrieved_sentences/Task3_CLIP_full_Knowledge_retrieved_sentences_NER_results.csv'
title = 'NER Evaluation for retrieved sentences on full_knowledge'

# Call the function and print the results
# accuracy, precision, recall, f1 = evaluate_ner_performance(data_df_path, ner_df_path, title)
evaluate_ner_performance_samplewise_retrieved(data_df_path, ner_df_path, title)
# # Output the results
# print(f"Accuracy: {accuracy}")
# print(f"Precision: {precision}")
# print(f"Recall: {recall}")
# print(f"F1 Score: {f1}")

Average Evaluation Results for NER Evaluation for retrieved sentences on full_knowledge:
  Average Accuracy: 0.4296
  Average Precision: 0.6293
  Average Recall: 0.4296
  Average F1 Score: 0.4954


In [8]:
ner_df_path = '/kaggle/input/business-json/Task_3_ner_results/Task_3_ner_results/Retrieved_sentences/Task3_CLIP_partial_Knowledge_retrieved_sentences_NER_results.csv'
title = 'NER Evaluation for retrieved sentences on partial_knowledge'

# Call the function and print the results
# accuracy, precision, recall, f1 = evaluate_ner_performance(data_df_path, ner_df_path, title)
evaluate_ner_performance_samplewise_retrieved(data_df_path, ner_df_path, title)
# # Output the results
# print(f"Accuracy: {accuracy}")
# print(f"Precision: {precision}")
# print(f"Recall: {recall}")
# print(f"F1 Score: {f1}")

Average Evaluation Results for NER Evaluation for retrieved sentences on partial_knowledge:
  Average Accuracy: 0.3994
  Average Precision: 0.6121
  Average Recall: 0.3994
  Average F1 Score: 0.4695


In [9]:
ner_df_path = '/kaggle/input/business-json/Task_3_ner_results/Task_3_ner_results/Retrieved_sentences/Task3_CLIP_label_retrieved_sentences_NER_results.csv'
title = 'NER Evaluation for retrieved sentences on label_knowledge'

# Call the function and print the results
# accuracy, precision, recall, f1 = evaluate_ner_performance(data_df_path, ner_df_path, title)
evaluate_ner_performance_samplewise_retrieved(data_df_path, ner_df_path, title)
# # Output the results
# print(f"Accuracy: {accuracy}")
# print(f"Precision: {precision}")
# print(f"Recall: {recall}")
# print(f"F1 Score: {f1}")

Average Evaluation Results for NER Evaluation for retrieved sentences on label_knowledge:
  Average Accuracy: 0.4009
  Average Precision: 0.6121
  Average Recall: 0.4009
  Average F1 Score: 0.4698
